In [11]:
import random
import os

import pandas as pd
import numpy as np
from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    f1_score,
    precision_score,
    recall_score,
)
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold #교차검증
from sklearn.model_selection import GridSearchCV #하이퍼파라미터 튜닝을 위한 불러오기
from sklearn.preprocessing import MinMaxScaler   #스케일링

In [12]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    
my_seed = 400
seed_everything(my_seed) # Seed 고정

In [21]:
df_train = pd.read_csv("C:/Users/Park Yoonseo/OneDrive/문서/LG Aimers 강의자료/train1.csv", encoding_errors='ignore') # 학습용 데이터
df_test = pd.read_csv("C:/Users/Park Yoonseo/OneDrive/문서/LG Aimers 강의자료/submission1.csv", encoding_errors='ignore') # 테스트 데이터(제출파일의 데이터)

In [22]:
def label_encoding(series: pd.Series) -> pd.Series:
    """범주형 데이터를 시리즈 형태로 받아 숫자형 데이터로 변환합니다."""

    my_dict = {}

    # 모든 요소를 문자열로 변환
    series = series.astype(str)

    for idx, value in enumerate(sorted(series.unique())):
        my_dict[value] = idx
    series = series.map(my_dict)

    return series

In [23]:
# 레이블 인코딩할 칼럼들
label_columns = [
    "customer_country",
    "business_subarea",
    "business_area",
    "business_unit",
    "customer_type",
    "enterprise",
    "customer_job",
    "inquiry_type",
    "product_category",
    "product_subcategory",
    "product_modelname",
    "customer_country.1",
    "customer_position",
    "response_corporate",
    "expected_timeline",
]

df_all = pd.concat([df_train[label_columns], df_test[label_columns]])

for col in label_columns:
    df_all[col] = label_encoding(df_all[col])

In [24]:
for col in label_columns:  
    df_train[col] = df_all.iloc[: len(df_train)][col]
    df_test[col] = df_all.iloc[len(df_train) :][col]

In [25]:
x_train, x_val, y_train, y_val = train_test_split(
    df_train.drop("is_converted", axis=1),
    df_train["is_converted"],
    test_size=0.2,
    shuffle=True,
    random_state=400,
)

In [26]:
# train

train_nan = x_train.copy()

zero_cols = ['com_reg_ver_win_rate',
             'historical_existing_cnt',
             'id_strategic_ver',
             'it_strategic_ver',
             'idit_strategic_ver']

for col in zero_cols:
    train_nan[col] = train_nan[col].fillna(0)

mean_cols = ['ver_win_rate_x','ver_win_ratio_per_bu']
for col in mean_cols:
    train_nan[col] = train_nan[col].fillna(train_nan[col].mean())
    
x_train = train_nan

# validation

val_nan = x_val.copy()

val_zero_cols = ['com_reg_ver_win_rate',
                 'historical_existing_cnt',
                 'id_strategic_ver',
                 'it_strategic_ver',
                 'idit_strategic_ver']

for col in val_zero_cols:
    val_nan[col] = val_nan[col].fillna(0)

val_mean_cols = ['ver_win_rate_x','ver_win_ratio_per_bu']
for col in val_mean_cols:
    val_nan[col] = val_nan[col].fillna(train_nan[col].mean())

x_val = val_nan

# test

test_nan = df_test.copy()

test_zero_cols = ['com_reg_ver_win_rate',
                  'historical_existing_cnt',
                  'id_strategic_ver',
                  'it_strategic_ver',
                  'idit_strategic_ver']

for col in test_zero_cols:
    test_nan[col] = test_nan[col].fillna(0)
    
test_mean_cols = ['ver_win_rate_x','ver_win_ratio_per_bu']
for col in test_mean_cols:
    test_nan[col] = test_nan[col].fillna(train_nan[col].mean())

df_test = test_nan

In [27]:
x_train = x_train.drop(['ver_win_rate_x','ver_win_ratio_per_bu'],axis=1)
x_val = x_val.drop(['ver_win_rate_x','ver_win_ratio_per_bu'],axis=1)
df_test = df_test.drop(['ver_win_rate_x','ver_win_ratio_per_bu'],axis=1)

In [28]:
model = DecisionTreeClassifier()

In [29]:
model.fit(x_train, y_train)

DecisionTreeClassifier()

In [30]:
def get_clf_eval(y_test, y_pred=None):
    confusion = confusion_matrix(y_test, y_pred, labels=[True, False])
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, labels=[True, False])
    recall = recall_score(y_test, y_pred)
    F1 = f1_score(y_test, y_pred, labels=[True, False])

    print("오차행렬:\n", confusion)
    print("\n정확도: {:.4f}".format(accuracy))
    print("정밀도: {:.4f}".format(precision))
    print("재현율: {:.4f}".format(recall))
    print("F1: {:.4f}".format(F1))

In [31]:
pred = model.predict(x_val.fillna(0))
get_clf_eval(y_val, pred)

오차행렬:
 [[  752   195]
 [  239 10674]]

정확도: 0.9634
정밀도: 0.7588
재현율: 0.7941
F1: 0.7761


In [32]:
# 예측에 필요한 데이터 분리
x_test = df_test.drop(["is_converted", "id"], axis=1)

In [33]:
test_pred = model.predict(x_test.fillna(0))
sum(test_pred) # True로 예측된 개수

2242

In [36]:
# 제출 데이터 읽어오기 (df_test는 전처리된 데이터가 저장됨)
df_sub = pd.read_csv("C:/Users/Park Yoonseo/OneDrive/문서/LG Aimers 강의자료/submission1.csv", encoding_errors='ignore')
df_sub["is_converted"] = test_pred

# 제출 파일 저장
df_sub.to_csv("C:/Users/Park Yoonseo/OneDrive/문서/LG Aimers 강의자료/submission1.csv", encoding_errors='ignore', index=False)

TypeError: NDFrame.to_csv() got an unexpected keyword argument 'encoding_errors'